# 导入模块

In [1]:
import numpy as np
import pandas as pd
import xlwings as xw
from openpyxl import load_workbook
import matplotlib.pyplot as plt
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False

import warnings
warnings.filterwarnings("ignore")
from jsonpath import jsonpath
import pymysql
from sqlalchemy import create_engine
import json
import random
import datetime
from datetime import timedelta ,time, datetime
from dateutil.relativedelta import relativedelta
import re
import os
import glob
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Week_Model, Data_Clean

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# 实例化All_Model类
all_models = All_Model()
week_models = Week_Model()
clean = Data_Clean()

# 链接数据库

In [2]:
def query(sql,
          host="rm-wz930e5269fur1ht1mo.mysql.rds.aliyuncs.com",
          user="wxz",
          password="5JRcY9SaiepVlIq7iuPo",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

# 查询数据

In [ ]:
sql = '''
select
om.id as order_id,
om.create_time, om.order_number, om.status, tprm.all_deposit, tprm.all_rental,tprm.purchase_amount, tod.new_actual_money, tod.sku_attributes, tpmn.name 机型,
top.total_describes, pa.name activity_name,tmu.true_name, tmu.id_card_num, om.user_mobile
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2 
,tps.deposit, tojo.app_type
from db_digua_business.t_order om
left join db_digua_business.t_postlease_receivables_monitoring tprm on tprm.order_id=om.id
left join db_digua_business.t_order_details tod on tod.order_id=om.id
-- 商品表
left join db_digua_business.t_product tp on tp.id = tod.product_id
left join db_digua_business.t_product_sku tps on tps.id=tod.sku_id
-- 商品型号
left join db_digua_business.t_product_model_number tpmn on tpmn.id=tp.model_number_id
-- 备注信息合并 
left join ( SELECT  t.order_id,JSON_ARRAYAGG(t.describes) as total_describes from db_digua_business.t_order_personnel t   GROUP BY 1 ) top on om.id = top.order_id 
-- 活动名称
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
-- 用户信息 
left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
-- 京东外部订单关联表
left join db_digua_business.t_order_jd_out_no tojo on tojo.order_id=om.id
where  date_format(om.create_time, '%Y-%m-%d')>='2025-06-01' 
and date_format(om.create_time, '%Y-%m-%d')<='2025-07-07' 
and om.order_type='JD_ORDER'
'''
df = query(sql)

# 清理数据

In [4]:
# 处理日期
df["下单日期"]=df["create_time"].dt.date
# df["下单日期"]=pd.to_datetime(df["下单日期"],errors="coerce")
df["下单月份"]=df["create_time"].dt.strftime('%Y-%m')


dict_status_code = {
    "订单取消": 1,
    "待支付": 2,
    "已退款": 3, 
    "待审核": 4,
    "待发货": 5,
    "待收货": 6,
    "租赁中": 7,
    "已完成": 8
}
df["状态编码"]=df["status2"].map(dict_status_code)


df.sort_values(by=["下单日期","状态编码"],inplace=True)


#删除重复单号
df.drop_duplicates(subset=["order_id"],inplace=True)
#删除身份证空值行
df.dropna(subset=["id_card_num"],axis=0,inplace=True)
#去刷单订单
df.drop(df[df['total_describes'].str.contains(pat='panli',regex=False)==True].index,inplace=True)
df.drop(df[df['activity_name']=="1000单秘密计划"].index,inplace=True)
df.drop(df[df['activity_name']=="1000单秘密计划-无优惠"].index,inplace=True)
df.drop(df[df['activity_name']=="1000单曙光计划"].index,inplace=True)
df.drop(df[df['activity_name']=="线下门店3个月试行"].index,inplace=True)
#删除身份证空值行
df.dropna(subset=["status2"],axis=0,inplace=True)


#删除重复订单
df.drop_duplicates(subset=["order_id"],inplace=True)
df.drop_duplicates(subset=["true_name","user_mobile","id_card_num","下单日期"],keep="last",inplace=True)


df.drop(df[df['true_name'].isin(["刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文"])].index,inplace=True)

df.loc[:,"内存"]=df.apply(lambda x: clean.getneicun(x["sku_attributes"]),axis=1)
df.loc[:,"租赁方案"]=np.where(df["sku_attributes"].str.contains('租完即送'),"租完即送","租完归还")


In [5]:
def zh(df):
    df.loc[:, '进件'] = np.where(df.status.isin([1,13]), 0, 1)
    df.loc[:, '出库'] = np.where(df.status.isin([2,3,4,5,6,8,15]), 1, 0)
    df_group = df.groupby('下单日期').agg({'order_number': 'count', '进件': 'sum', '出库': 'sum'}).rename(columns={'order_number': '去重订单数'}).T
    df_group.loc[:, '月合计'] = df_group.sum(axis=1)
    df_group.insert(15, '15日合计', df_group.iloc[:, :15].sum(axis=1))
    df_group = df_group.T
    df_group.loc[:, '进件出库率'] = (df_group.出库/df_group.进件).map(lambda x: format(x, '.2%'))
    df_group.loc[:, '订单出库率'] = (df_group.出库/df_group.去重订单数).map(lambda x: format(x, '.2%'))
    df_groups = df_group.T
    return df_groups
df_all = zh(df)
df_all.loc['京东商城',:] = np.nan
df_jd = df[df.app_type=='JD_APP']
df_jd_all = zh(df_jd)
df_jd_all.loc['京东金融',:] = np.nan
df_fjd = df[df.app_type!='JD_APP']
df_fjd_all = zh(df_fjd)
df_groups = pd.concat([df_all, df_jd_all, df_fjd_all])


In [ ]:
df_new = df[(df.出库==1)&(df.purchase_amount>0)]

df_group = df_new.groupby(['下单月份', '租赁方案']).agg({'new_actual_money': 'mean', 'all_rental': 'mean', 'deposit': 'mean', 'purchase_amount': 'mean'}).rename(columns={'new_actual_money': '签约价均值', 'all_rental': '总租金均值', 'deposit': '押金均值', 'purchase_amount': '采购金额均值'})
col_list = df_group.columns.to_list()
for col in col_list:
    df_group.loc[:, f'{col}'] = np.round(df_group[f'{col}'], 0)
df_group.loc[:, '签约价租售比'] = (df_group.签约价均值/df_group.押金均值).map(lambda x: format(x, '.2%'))
df_group.loc[:, '总租金租售比'] = (df_group.总租金均值/df_group.押金均值).map(lambda x: format(x, '.2%'))


In [7]:
df.loc[:, '机型内存'] = df.机型+'_'+df.内存
def sku(df, key):
    df_new = df[df.purchase_amount>0]
    df_group = df.groupby(f'{key}').agg({'进件': 'sum', '出库': 'sum'})
    df_new_group = df_new.groupby(f'{key}').agg({'new_actual_money': 'mean', 'all_rental': 'mean', 'deposit': 'mean', 'purchase_amount': 'mean'}).rename(columns={'new_actual_money': '签约价均值', 'all_rental': '总租金均值', 'deposit': '押金均值', 'purchase_amount': '采购金额均值'})
    df_group = df_group.merge(df_new_group, on=f'{key}', how='left')
    df_group = df_group.fillna(0)
    df_group.loc['汇总', :] = df_group.sum(axis=0)
    df_group.loc[:, '机型占比'] = (df_group.出库/df_group.loc['汇总', '出库']).map(lambda x: format(x, '.2%'))
    df_group.loc[:, '签约价租售比'] = (df_group.签约价均值/df_group.押金均值).map(lambda x: format(x, '.2%'))
    df_group.loc[:, '总租金租售比'] = (df_group.总租金均值/df_group.押金均值).map(lambda x: format(x, '.2%'))
    df_group = df_group.sort_values(by='进件', ascending=False)
    return df_group
df_sku = sku(df, '机型内存')
df_spu = sku(df, '机型')

In [8]:
with pd.ExcelWriter('D:/需求/瑞彤需求/京东复盘转化数据.xlsx', engine='xlsxwriter') as writer:
    df_groups.to_excel(writer, sheet_name='京东目标转化率')
    df_group.to_excel(writer, sheet_name='京东租售比')
    df_sku.to_excel(writer, sheet_name='京东sku')
    df_spu.to_excel(writer, sheet_name='京东spu')
